# 🚦 YOLO Detection Server V2
### localhost.run (FREE SSH Tunnel) + Auto-register URL

**Flow:**
1. Kaggle starts Flask API on port 5000
2. SSH tunnel via localhost.run → Public URL
3. Auto-POST URL to Node.js backend `/api/yolo/url`
4. Backend sends images via HTTP to this URL `/detect`

In [ ]:
# ═══════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION - EDIT THIS!
# ═══════════════════════════════════════════════════════════

# Your Node.js backend URL (this server will POST its public URL here)
BACKEND_URL = 'http://YOUR_NODEJS_SERVER:3000'

# Heartbeat interval (seconds) - send URL to backend periodically
HEARTBEAT_INTERVAL = 60

print(f"📌 Backend: {BACKEND_URL}")
print(f"💓 Heartbeat: every {HEARTBEAT_INTERVAL}s")

In [ ]:
# Cell 2: Install Dependencies
!pip install ultralytics flask flask-cors opencv-python-headless pillow requests --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import os
if not os.path.exists('yolov5'):
    !git clone --depth 1 https://github.com/ultralytics/yolov5.git 2>/dev/null

# Patch yolov5 to avoid scipy/seaborn issues
for f, old, new in [
    ('yolov5/utils/plots.py', 'import seaborn as sn', '# seaborn'),
    ('yolov5/utils/plots.py', 'from scipy.ndimage.filters import gaussian_filter1d', '# scipy'),
    ('yolov5/models/yolo.py', 'from utils.plots import feature_visualization', '# plots')
]:
    try:
        with open(f, 'r') as x: c = x.read()
        with open(f, 'w') as x: x.write(c.replace(old, new))
    except: pass

# Generate SSH key for localhost.run
!mkdir -p ~/.ssh
!ssh-keygen -t rsa -N '' -f ~/.ssh/id_rsa -q <<< y 2>/dev/null || true

print("✅ Dependencies installed")

In [ ]:
# Cell 3: Load Models
import torch, sys, os
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Device: {device}")

vehicle_model = traffic_light_model = lp_detector_model = lp_ocr_model = None

# Vehicle
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print("✅ Vehicle")
except Exception as e: print(f"⚠️ Vehicle: {e}")

# Traffic Light
try:
    p = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-dentinhieu-best-new.pt'
    if os.path.exists(p):
        traffic_light_model = YOLO(p).to(device)
        print("✅ Traffic Light")
except Exception as e: print(f"⚠️ TL: {e}")

# License Plate
try:
    lp_det = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_detector.pt'
    lp_ocr = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_ocr.pt'
    if os.path.exists(lp_det):
        if 'yolov5' not in sys.path: sys.path.insert(0, os.path.abspath('yolov5'))
        lp_detector_model = torch.load(lp_det, map_location=device, weights_only=False)['model'].float().eval()
        lp_ocr_model = torch.load(lp_ocr, map_location=device, weights_only=False)['model'].float().eval()
        if device == 'cuda': lp_detector_model, lp_ocr_model = lp_detector_model.cuda(), lp_ocr_model.cuda()
        print("✅ LP")
except Exception as e: print(f"⚠️ LP: {e}")

print(f"\n📊 V:{'✅' if vehicle_model else '❌'} TL:{'✅' if traffic_light_model else '❌'} LP:{'✅' if lp_detector_model else '❌'}")

In [ ]:
# Cell 4: Flask API Server
import cv2, numpy as np, time, threading, base64, io, requests, re, json
from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
from datetime import datetime

app = Flask(__name__)
CORS(app)
app.config['MAX_CONTENT_LENGTH'] = 50 * 1024 * 1024

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
CONFIDENCE = 0.5
camera_trackers = {}
public_url = None
running = True
request_count = 0

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def register_url(url):
    """POST URL to backend /api/yolo/url"""
    if 'YOUR_NODEJS' in BACKEND_URL: return False
    try:
        resp = requests.post(f"{BACKEND_URL}/api/yolo/url", json={'url': url}, timeout=10)
        if resp.ok:
            log(f"✅ Registered with backend")
            return True
        log(f"⚠️ Registration failed: {resp.status_code}")
    except Exception as e:
        log(f"❌ Registration error: {e}")
    return False

def heartbeat_loop(url, interval):
    """Periodically send URL to backend"""
    while running:
        time.sleep(interval)
        try:
            register_url(url)
            log(f"💓 Heartbeat | Requests: {request_count}")
        except: pass

@app.route('/health')
def health():
    return jsonify({
        'status': 'ok',
        'models': {
            'vehicle': vehicle_model is not None,
            'traffic_light': traffic_light_model is not None,
            'license_plate': lp_detector_model is not None
        },
        'device': device,
        'public_url': public_url,
        'request_count': request_count
    })

@app.route('/detect', methods=['POST'])
def detect():
    global request_count
    request_count += 1
    
    try:
        # Parse request (multipart or JSON)
        if 'image' in request.files:
            img_bytes = request.files['image'].read()
            camera_id = request.form.get('camera_id', 'unknown')
            track_line_y = float(request.form.get('track_line_y', 50))
            created_at = float(request.form.get('created_at', time.time()*1000))
        elif request.is_json:
            data = request.get_json()
            img_bytes = base64.b64decode(data.get('image', ''))
            camera_id = data.get('camera_id', 'unknown')
            track_line_y = data.get('track_line_y', 50)
            created_at = data.get('created_at', time.time()*1000)
        else:
            return jsonify({'error': 'No image'}), 400
        
        # Decode image
        img = Image.open(io.BytesIO(img_bytes))
        frame = np.array(img)
        if len(frame.shape) == 2: frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        elif frame.shape[2] == 4: frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
        else: frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        h, w = frame.shape[:2]
        result = {'camera_id': camera_id, 'created_at': created_at, 'image_dimensions': {'width': w, 'height': h}}
        
        # Init tracker
        if camera_id not in camera_trackers:
            camera_trackers[camera_id] = {
                'tracks': {}, 'counted': {},
                'counts_up': {v:0 for v in VEHICLE_CLASSES},
                'counts_down': {v:0 for v in VEHICLE_CLASSES},
                'total_up': 0, 'total_down': 0
            }
        tr = camera_trackers[camera_id]
        
        # Vehicle Detection
        if vehicle_model:
            t0 = time.time()
            dets, tracks, vcounts, crossings = [], {}, {v:0 for v in VEHICLE_CLASSES}, []
            ly = int(h * track_line_y / 100)
            
            for r in vehicle_model.track(frame, persist=True, verbose=False):
                for b in r.boxes:
                    cls = vehicle_model.names[int(b.cls[0])]
                    if cls not in VEHICLE_CLASSES or float(b.conf[0]) < CONFIDENCE: continue
                    x1,y1,x2,y2 = map(int, b.xyxy[0])
                    cx, cy = (x1+x2)//2, (y1+y2)//2
                    det = {'class':cls, 'type':'vehicle', 'confidence':float(b.conf[0]), 'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}}
                    if hasattr(b,'id') and b.id is not None:
                        tid = int(b.id[0]); det['id'] = tid
                        tracks[tid] = {'pos': (cx,cy), 'time': created_at, 'class': cls}
                    dets.append(det); vcounts[cls] += 1
            
            # Counting
            for tid, info in tracks.items():
                if tid not in tr['tracks']: tr['tracks'][tid] = []
                if tr['tracks'][tid]:
                    py, cy = tr['tracks'][tid][-1]['pos'][1], info['pos'][1]
                    d = 1 if py <= ly < cy else (-1 if py >= ly > cy else 0)
                    if d and f"{tid}_{d}" not in tr['counted']:
                        tr['counted'][f"{tid}_{d}"] = True
                        if d == 1: tr['counts_down'][info['class']] += 1; tr['total_down'] += 1
                        else: tr['counts_up'][info['class']] += 1; tr['total_up'] += 1
                        crossings.append({'id': tid, 'direction': d})
                tr['tracks'][tid].append({'pos': info['pos'], 'time': info['time'], 'class': info['class']})
                tr['tracks'][tid] = tr['tracks'][tid][-30:]
            
            result['vehicle'] = {
                'detections': dets,
                'inference_time': (time.time()-t0)*1000,
                'vehicle_count': {
                    'total_up': tr['total_up'], 'total_down': tr['total_down'],
                    'by_type_up': tr['counts_up'].copy(), 'by_type_down': tr['counts_down'].copy(),
                    'current': vcounts
                },
                'new_crossings': crossings
            }
        
        # Traffic Light
        if traffic_light_model:
            t0 = time.time(); tl_dets = []; status = None; mx = 0
            for r in traffic_light_model(frame, verbose=False):
                for b in r.boxes:
                    cf = float(b.conf[0])
                    if cf < 0.4: continue
                    x1,y1,x2,y2 = map(int, b.xyxy[0])
                    cn = traffic_light_model.names[int(b.cls[0])]
                    tl_dets.append({'class':cn,'confidence':cf,'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
                    if cf > mx: mx, status = cf, cn
            result['traffic_light'] = {'detections': tl_dets, 'traffic_status': status, 'inference_time': (time.time()-t0)*1000}
        
        return jsonify(result)
    
    except Exception as e:
        import traceback; traceback.print_exc()
        return jsonify({'error': str(e)}), 500

@app.route('/detect/lp', methods=['POST'])
def detect_lp():
    if not lp_detector_model:
        return jsonify({'error': 'LP model not loaded'}), 503
    # Simplified - return empty for now
    return jsonify({'license_plates': {}, 'inference_time': 0})

# Start Flask in background
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
print("✅ Flask API on port 5000")
time.sleep(2)

In [ ]:
# Cell 5: localhost.run SSH Tunnel + Auto-Register (RUNS FOREVER)
import subprocess, re, threading, time, requests

public_url = None
registered = False

def on_url_found(url):
    """Called when tunnel URL is extracted"""
    global public_url, registered
    public_url = url
    
    print(f"\n{'═'*60}")
    print(f"🌐 PUBLIC URL: {url}")
    print(f"{'═'*60}")
    print(f"\n📋 Endpoints:")
    print(f"   GET  {url}/health")
    print(f"   POST {url}/detect")
    print(f"   POST {url}/detect/lp")
    print(f"{'═'*60}\n")
    
    # Auto-register with backend
    if 'YOUR_NODEJS' not in BACKEND_URL:
        if register_url(url):
            registered = True
            # Start heartbeat thread
            threading.Thread(target=heartbeat_loop, args=(url, HEARTBEAT_INTERVAL), daemon=True).start()
            print("💓 Heartbeat started")
    else:
        print("⚠️ Set BACKEND_URL in Cell 1 to enable auto-registration")

print("🚀 Starting localhost.run SSH tunnel...")
print("═" * 60)

# Run SSH tunnel to localhost.run
process = subprocess.Popen(
    ['ssh', '-o', 'StrictHostKeyChecking=no', '-R', '80:localhost:5000', 'localhost.run'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
)

# Parse output for URL
for line in process.stdout:
    print(line, end='')
    
    # Look for the tunnel URL (format: https://xxx.lhr.life or similar)
    match = re.search(r'https://[a-zA-Z0-9-]+\.[a-z]+\.life', line)
    if match and not public_url:
        on_url_found(match.group(0))
    
    # Also check for alternative URL formats
    match2 = re.search(r'https://[a-zA-Z0-9]+\.lhr\.life', line)
    if match2 and not public_url:
        on_url_found(match2.group(0))